### SVM 处理 20180127

#### 总结 20180209
* 适用：小样本、分类问题
* 优点：
* 缺点：
* 重要参数：

#### 完成：
* 得分0.80382
* 默认核函数下，参数C、Gamma调式，但Gamma规律不明确
* 大致过程，具体求解还需要

#### 暂确定还需要：
* Gamma 及其它参数调式
* 核函数选择
* 如何避免局部最优情况

In [4]:
#数据预处理--同feature

#导入必要的包
import pandas as pd
import numpy as np
from pandas import DataFrame,Series

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn import cross_validation
from sklearn.svm import SVC

#读入文件
data_train = pd.read_csv("c:/Users/chai/Downloads/train.csv")
data_test = pd.read_csv("c:/Users/chai/Downloads/test.csv")

# For Pre-Processing, combine train/test to simultaneously apply transformations
Survived = data_train['Survived'].copy()
train_df = data_train.drop('Survived', axis=1)

traindex = train_df.index
testdex = data_test.index


data_all = pd.concat([data_train,data_test],ignore_index = True)
data_all.describe()

c:\users\chai\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [31]:
#直接计算衍生变量
data_all['FamilySize'] = data_all.SibSp + data_all.Parch +1
data_all['NameLength'] = data_all.Name.apply(len)
data_all['IsAlone'] = 0
data_all['IsAlone'][data_all['FamilySize'] == 1] = 1   
## 或 df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1
data_all['TicketLen'] = data_all.Ticket.apply(len)
data_all.head()


df = data_all

df['Title']=0
df['Title']=df.Name.str.extract('([A-Za-z]+)\.') #lets extract the Salutations
df['Title'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don','Dona'],
                          ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr','Mr'],inplace=True)

df.loc[(df.Age.isnull())&(df.Title=='Mr'),'Age']= 33
df.loc[(df.Age.isnull())&(df.Title=='Mrs'),'Age']=36
df.loc[(df.Age.isnull())&(df.Title=='Master'),'Age']=5
df.loc[(df.Age.isnull())&(df.Title=='Miss'),'Age']=22
df.loc[(df.Age.isnull())&(df.Title=='Rare'),'Age']=46

df.head()

df.Embarked = df.Embarked.fillna(df.Embarked.mode().iloc[0])
# Continuous Variable
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())

from sklearn import preprocessing
for col in ['Fare','Age','NameLength']:
    transf = df[col].values.reshape(-1,1)
    scaler = preprocessing.StandardScaler().fit(transf)
    df[col] = scaler.transform(transf)

#增加标准化：
for col in ['NameLength','TicketLen','Parch','FamilySize','SibSp']:
    transf = df[col].values.reshape(-1,1)
    scaler = preprocessing.StandardScaler().fit(transf)
    df[col] = scaler.transform(transf)

    
df['cabinfix']=1
df['cabinfix'][df['Cabin'].isnull()] = 0
df.head()


#哑变量相关：Sex\ Pclass \ Embarked ,

dum_Embarked = pd.get_dummies(data_all.Embarked,prefix = 'embarked')
dum_Sex = pd.get_dummies(data_all.Sex,prefix = 'sex')
dum_Pclass = pd.get_dummies(data_all.Pclass,prefix = 'pclass')
dum_nametitle = pd.get_dummies(df['Title'],prefix = 'title')
df = pd.concat([df,dum_Embarked,dum_Sex,dum_Pclass,dum_nametitle],axis=1)

df = df.drop(['Name','Sex','Embarked','Title','Pclass','Ticket','Cabin'],axis = 1)
df.head()

c:\users\chai\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\chai\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
c:\users\chai\appdata\local\programs\python\python35\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)
c:\users\chai\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a

,Age,Fare,Parch,PassengerId,SibSp,Survived,FamilySize,NameLength,IsAlone,TicketLen,...,sex_female,sex_male,pclass_1,pclass_2,pclass_3,title_Master,title_Miss,title_Mr,title_Mrs,title_Other
0,-0.603970,-0.503595,-0.445,1,0.481288,0.0,0.073352,-0.434672,0,0.798033,...,0,1,0,0,1,0,0,1,0,0
1,0.608524,0.734503,-0.445,2,0.481288,1.0,0.073352,2.511806,0,0.436821,...,1,0,1,0,0,0,0,0,1,0
2,-0.300847,-0.490544,-0.445,3,-0.479087,1.0,-0.558346,-0.539904,1,3.326516,...,1,0,0,0,1,0,1,0,0,0
3,0.381181,0.382925,-0.445,4,0.481288,1.0,0.073352,1.775186,0,-0.285603,...,1,0,1,0,0,0,0,0,1,0
4,0.381181,-0.488127,-0.445,5,-0.479087,0.0,-0.558346,-0.329441,1,-0.285603,...,0,1,0,0,1,0,0,1,0,0


In [26]:
#df.describe().to_csv("D:/work/others/kaggle/数据描述统计0127用于svm优化.csv", index=False)

In [32]:
#拆分成两个数据集：
df_train_fix = df[df.Survived.notnull()]
df_train_fix.describe()
var_to_use = 'Age|Fare|Parch|SibSp|FamilySize|Namelength|TicketLen|cabinfix|embarked_.*|sex_.*|pclass_.*|title_.*'
all_data = df_train_fix.filter(regex=var_to_use)
X = all_data.as_matrix()[:,:]
y = df_train_fix['Survived'].as_matrix()
df_train_fix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 24 columns):
Age             891 non-null float64
Fare            891 non-null float64
Parch           891 non-null float64
PassengerId     891 non-null int64
SibSp           891 non-null float64
Survived        891 non-null float64
FamilySize      891 non-null float64
NameLength      891 non-null float64
IsAlone         891 non-null int64
TicketLen       891 non-null float64
cabinfix        891 non-null int64
embarked_C      891 non-null uint8
embarked_Q      891 non-null uint8
embarked_S      891 non-null uint8
sex_female      891 non-null uint8
sex_male        891 non-null uint8
pclass_1        891 non-null uint8
pclass_2        891 non-null uint8
pclass_3        891 non-null uint8
title_Master    891 non-null uint8
title_Miss      891 non-null uint8
title_Mr        891 non-null uint8
title_Mrs       891 non-null uint8
title_Other     891 non-null uint8
dtypes: float64(8), int64(3), uint8(13)

In [51]:
#网格搜索
# list(np.arange(0.1,3.2,0.2))
# list(np.arange(0.01,0.5,0.02))
from sklearn.grid_search import GridSearchCV
param_test1 ={'C':[3.5,4],'gamma':list(np.arange(0.01,0.5,0.02))}
gsearch1= GridSearchCV(estimator = SVC(random_state = 1),
                       param_grid =param_test1,scoring='accuracy',cv=10)  
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_  

([mean: 0.83165, std: 0.02888, params: {'gamma': 0.01, 'C': 3.5},
  mean: 0.83726, std: 0.03220, params: {'gamma': 0.029999999999999999, 'C': 3.5},
  mean: 0.83502, std: 0.03576, params: {'gamma': 0.049999999999999996, 'C': 3.5},
  mean: 0.82828, std: 0.04009, params: {'gamma': 0.069999999999999993, 'C': 3.5},
  mean: 0.82379, std: 0.03862, params: {'gamma': 0.089999999999999983, 'C': 3.5},
  mean: 0.82379, std: 0.04414, params: {'gamma': 0.10999999999999997, 'C': 3.5},
  mean: 0.81930, std: 0.03843, params: {'gamma': 0.12999999999999998, 'C': 3.5},
  mean: 0.81145, std: 0.03976, params: {'gamma': 0.14999999999999999, 'C': 3.5},
  mean: 0.80920, std: 0.03743, params: {'gamma': 0.16999999999999998, 'C': 3.5},
  mean: 0.80920, std: 0.03600, params: {'gamma': 0.18999999999999997, 'C': 3.5},
  mean: 0.80584, std: 0.03597, params: {'gamma': 0.20999999999999996, 'C': 3.5},
  mean: 0.80247, std: 0.03490, params: {'gamma': 0.22999999999999998, 'C': 3.5},
  mean: 0.80247, std: 0.03553, params: 

### 运行调式记录
20180123 
* 首次无优化+ 默认参数（random_state = 1）,交叉验证score 0.856933010481568：  0.79904
* C=2.2，0.79425
* c1.7,gamma 0.1   线上 0.78468

20180127
* {'C': 1.7, 'gamma': 0.1}, 0.8327721661054994)
* 增加两个变量的标准化之后  {'C': 2.2, 'gamma': 'auto'}, 0.8361391694725028) —— 0.79425
* 增加五个变量的标准化之后  {'C': 2.2, 'gamma': 'auto'}, 0.835016835016835) 
* 固定gamma找c  {'C': 1.9000000000000004, 'gamma': 'auto'}, 0.835016835016835)
* {'C':[1.8,1.9,1.901,2],'gamma':['auto',0.001,0.002,0.003]} ==》 {'C': 1.8, 'gamma': 'auto'}, 0.835016835016835) ===》 C 与 gamma相互还是有影响 —— 0.79904
* param_test1 ={'C':[1.9,2,3,4,5,10],'gamma':list(np.arange(0.01,0.5,0.02))}　＝》 {'C': 4, 'gamma': 0.0299999},0.8372615039281706)===》 C之前应是陷入局部最小了？ —— 0.80382
* {'C':[1.9,2,3.5,4,4.5,5,8],'gamma':['auto',0.01,0.03,0.029,0.04,0.05,0.008]} => {'C': 3.5, 'gamma': 0.03}, 0.8372615039281706) => 0.80382

In [47]:
#预测评价。
df_test_fix = df[df.Survived.isnull()]
test_data = df_test_fix.filter(regex= var_to_use)
test_X = test_data.as_matrix()[:,:]
predict_y = gsearch1.predict(test_X)
result = pd.DataFrame({'PassengerId':df_test_fix['PassengerId'].as_matrix(), 'Survived':predict_y.astype(np.int32)})
result.head(30)
result.to_csv("D:/work/others/kaggle/20180127增加5个标准化girdserch_acc_C3.4gamma_0.03.csv", index=False)

In [19]:
## 开始svm  baseline
from sklearn import cross_validation
from sklearn import svm
clf = svm.SVC(random_state=1)
clf.fit(X,y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=1, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
print(cross_validation.cross_val_score(clf, X, y, cv=10))

[ 0.81111111  0.84444444  0.7752809   0.87640449  0.83146067  0.80898876
  0.83146067  0.79775281  0.87640449  0.85227273]


In [20]:
clf.support_vectors_

array([[ 1.82101732,  0.35899728,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [-2.11958692, -0.23628556,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.75553161, -0.48812669,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       ..., 
       [-1.96802522, -0.42851043,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       [-1.13443586, -0.50407824,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.3008465 , -0.06371879,  0.        , ...,  1.        ,
         0.        ,  0.        ]])

In [22]:
clf.n_support_

array([215, 210])

#### 问题列表
* 映射到高维空间的核函数是怎么选择的？怎么保证映射后就是线性可分的？
> 分为：线性可分、不可分等情形，对应取核函数。sklearn上默认是rbf。其它待查。
* 超平面是怎么算出来的？
* 有哪些参数需要调整，如何调整。
> rbf核函数中C和gamma ：
>SVM模型有两个非常重要的参数C与gamma。其中 C是惩罚系数，即对误差的宽容度。c越高，说明越不能容忍出现误差,容易过拟合。C越小，容易欠拟合。C过大或过小，泛化能力变差

> gamma是选择RBF函数作为kernel后，该函数自带的一个参数。隐含地决定了数据映射到新的特征空间后的分布，gamma越大，支持向量越少，gamma值越小，支持向量越多。支持向量的个数影响训练与预测的速度。

> C可能陷入局部最小，调式时候选值要有跳跃. Gamma，暂没发现规律,取值较小。

* 交叉验证调参时，评价指标用acc　和　用roc 有区别。

#### 参考资料

* [sklearn中SVM调参说明及经验总结](https://xijunlee.github.io/2017/03/29/sklearn%E4%B8%ADSVM%E8%B0%83%E5%8F%82%E8%AF%B4%E6%98%8E%E5%8F%8A%E7%BB%8F%E9%AA%8C%E6%80%BB%E7%BB%93/)
* 统计学习方法 李航
* [sigma-gamma](http://haohanw.blogspot.com/2014/03/ml-how-sigma-matters-in-svm-rbf-kernel.html) 